# Chapter 5: Support Vector Machines

Support Vector Machines (SVM) merupakan salah satu algoritma pembelajaran mesin yang dikenal memiliki performa tinggi serta fleksibilitas yang luas. Algoritma ini tidak hanya digunakan untuk permasalahan klasifikasi, tetapi juga dapat diterapkan pada regresi dan pendeteksian anomali.

Keunggulan utama SVM terletak pada kemampuannya membangun decision boundary dengan margin maksimum. Dengan margin yang lebar, model cenderung memiliki kemampuan generalisasi yang lebih baik ketika dihadapkan pada data baru yang belum pernah dilihat sebelumnya.

Pada chapter ini, pembahasan difokuskan pada:
- konsep large margin classification,
- perbedaan antara hard margin dan soft margin,
- SVM linear dan non-linear,
- penggunaan kernel trick,
- penerapan SVM untuk regresi,
- serta gambaran konseptual dan matematis dari mekanisme kerja SVM.

Pemahaman terhadap SVM menjadi sangat penting karena konsep margin dan kernel merupakan fondasi bagi banyak metode Machine Learning lanjutan.


## 1. Linear SVM Classification

Inti dari algoritma SVM adalah prinsip large margin classification. Alih-alih hanya memisahkan kelas secara benar, SVM berusaha mencari decision boundary yang berada sejauh mungkin dari titik data terdekat dari masing-masing kelas.

Decision boundary dengan margin yang besar umumnya lebih stabil dan memiliki tingkat generalisasi yang lebih baik. Titik-titik data yang berada tepat pada batas margin disebut sebagai support vectors.

Menariknya, hanya support vectors inilah yang secara langsung menentukan posisi dan orientasi decision boundary. Titik data lain yang berada jauh dari margin tidak memengaruhi bentuk boundary.


### 1.1 Sensitivity to Feature Scales

SVM sangat peka terhadap perbedaan skala fitur. Apabila satu fitur memiliki rentang nilai yang jauh lebih besar dibandingkan fitur lainnya, maka decision boundary dapat menjadi tidak seimbang dan bias terhadap fitur tersebut.

Oleh karena itu, proses feature scaling, seperti standardisasi menggunakan StandardScaler, hampir selalu menjadi langkah wajib sebelum melatih model SVM.


## 2. Soft Margin Classification

Dalam kondisi ideal, seluruh data dapat dipisahkan secara sempurna menggunakan decision boundary linear (hard margin). Namun pada data dunia nyata, kondisi ini jarang terjadi karena:
- data sering kali tidak sepenuhnya linearly separable,
- adanya noise dan outlier.

Untuk mengatasi permasalahan tersebut, SVM menerapkan konsep soft margin, yaitu mengizinkan sejumlah pelanggaran margin demi memperoleh model yang lebih robust.

Trade-off antara margin yang lebar dan jumlah kesalahan dikendalikan oleh hyperparameter C.
- Nilai C kecil membuat model lebih toleran terhadap kesalahan dan menghasilkan margin yang lebih lebar.
- Nilai C besar memaksa model mengklasifikasikan data seakurat mungkin, tetapi meningkatkan risiko overfitting.


## 3. Linear SVM Classification with Scikit-Learn

Pada bagian ini, dilakukan pelatihan Linear SVM untuk kasus binary classification menggunakan dataset Iris. Tujuan model adalah mengidentifikasi apakah sebuah bunga termasuk spesies Iris virginica atau bukan.

Dataset difokuskan pada dua fitur, yaitu panjang dan lebar kelopak bunga, agar decision boundary dapat dianalisis secara lebih intuitif.


In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.float64)  # Iris virginica


In [ ]:
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])

svm_clf.fit(X, y)


Pipeline di atas memastikan bahwa data mengalami proses standardisasi sebelum digunakan oleh model Linear SVM. Penggunaan loss function hinge sesuai dengan formulasi klasik SVM yang bertujuan memaksimalkan margin.


### 3.1 Making Predictions

Setelah model dilatih, model dapat digunakan untuk melakukan prediksi pada data baru. Output prediksi berupa label kelas, bukan probabilitas.


In [ ]:
svm_clf.predict([[5.5, 1.7]])


Berbeda dengan Logistic Regression, SVM tidak secara langsung menghasilkan probabilitas kelas. Output prediksi menunjukkan kelas hasil keputusan model.


## 4. Nonlinear SVM Classification

Tidak semua dataset dapat dipisahkan menggunakan decision boundary linear. Pada kondisi tersebut, Linear SVM tidak mampu memberikan hasil yang memadai.

Untuk mengatasi keterbatasan ini, SVM dapat diperluas menjadi model non-linear dengan cara memetakan data ke ruang berdimensi lebih tinggi, di mana data menjadi linearly separable.


### 4.1 Polynomial Features

Pendekatan sederhana untuk menangani non-linearitas adalah dengan menambahkan fitur polinomial. Dengan menambahkan kombinasi non-linear dari fitur asli, model linear dapat mempelajari boundary yang non-linear di ruang input.


In [ ]:
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=10, loss="hinge"))
])

polynomial_svm_clf.fit(X, y)


Pendekatan ini efektif untuk menangani data non-linear, namun jumlah fitur dapat meningkat sangat cepat seiring bertambahnya derajat polinomial.


## 5. The Kernel Trick (Pengantar)

Untuk menghindari eksplosinya jumlah fitur akibat penambahan fitur polinomial secara eksplisit, SVM memanfaatkan teknik yang dikenal sebagai kernel trick.

Kernel trick memungkinkan perhitungan hubungan di ruang berdimensi tinggi tanpa perlu benar-benar melakukan transformasi eksplisit ke ruang tersebut, sehingga tetap efisien secara komputasi.


## 6. Polynomial Kernel

Polynomial kernel memungkinkan SVM mempelajari decision boundary non-linear secara efisien melalui kernel trick. Kernel ini memiliki beberapa hyperparameter penting, seperti degree, coef0, dan C, yang memengaruhi kompleksitas model.


In [ ]:
from sklearn.svm import SVC

poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C=5)
poly_kernel_svm_clf.fit(X, y)


## 7. Gaussian Radial Basis Function (RBF) Kernel

RBF kernel merupakan salah satu kernel paling populer dalam SVM. Kernel ini memungkinkan pembentukan decision boundary yang sangat fleksibel dan kompleks dengan mengukur tingkat kemiripan antar instance data.


In [ ]:
rbf_kernel_svm_clf = SVC(kernel="rbf", gamma=0.5, C=1)
rbf_kernel_svm_clf.fit(X, y)


## 8. Support Vector Machine for Regression (SVR)

Selain klasifikasi, SVM juga dapat digunakan untuk regresi. Pendekatan ini dikenal sebagai Support Vector Regression (SVR), yang berfokus pada pencarian fungsi regresi yang berada dalam batas toleransi tertentu terhadap data training.


In [ ]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y.ravel())


In [ ]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y.ravel())


## Closing Summary (Chapter 5)

Chapter ini membahas Support Vector Machines sebagai algoritma yang kuat dan serbaguna. Konsep utama yang dipelajari meliputi large margin classification, soft margin, penggunaan kernel untuk menangani non-linearitas, serta penerapan SVM pada klasifikasi dan regresi.

Pemahaman terhadap margin, kernel, dan regularization menjadi bekal penting untuk memahami model Machine Learning lanjutan pada chapter berikutnya.
